# K-fold Cross Validation 을 이용한 집값 예측 

In [33]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split


In [34]:
data = pd.read_csv('data/reg.csv',index_col = 0)
data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,Price
0,0.034633,0.206919,0.137057,0.540526,0.193941,0.699239,0.630532,0.239410,0.027375,0.209857,0.347609,0.996394,0.102644,0.422222
1,0.028920,0.014315,0.276113,0.255945,0.618886,0.555407,0.782263,0.482977,0.103031,0.106690,0.520776,0.996650,0.187120,0.368889
2,0.020627,0.033230,0.281116,0.525591,0.165269,0.624102,0.586005,0.272713,0.036010,0.106986,0.595301,0.983284,0.084079,0.660000
3,0.022749,0.033801,0.125044,0.263253,0.251509,0.658532,0.432160,0.344932,0.150018,0.068317,0.651297,0.989989,0.015990,0.631111
4,0.022148,0.029374,0.121057,0.521126,0.399670,0.448086,0.520158,0.495342,0.104383,0.069360,0.560116,0.998723,0.092782,0.693333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.028702,0.019528,0.455716,0.097575,0.576859,0.588769,0.654701,0.188444,0.007595,0.165409,0.736795,0.982923,0.219891,0.386667
502,0.034217,0.009498,0.490485,0.372934,0.532351,0.580505,0.750547,0.143776,0.051186,0.166428,0.867950,0.995114,0.207453,0.346667
503,0.035352,0.021495,0.423918,0.397988,0.349407,0.610529,0.907637,0.087385,0.083448,0.164870,0.782704,0.995791,0.094044,0.420000
504,0.026182,0.028603,0.443442,0.509663,0.229142,0.667841,0.867135,0.236241,0.016177,0.167554,0.749186,0.986855,0.107399,0.377778


In [35]:
x = data.drop('Price',axis=1).to_numpy()
y = data['Price'].to_numpy().reshape(-1,1)
print(x.shape)
print(y.shape)

(506, 13)
(506, 1)


In [47]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(354, 13)
(354, 1)
(152, 13)
(152, 1)


In [42]:
class my_ds(Dataset):
    def __init__(self, x,y):
        self.x = torch.FloatTensor(x)
        self.y = torch.FloatTensor(y)
        self.len = self.y.size()[0]

    def __getitem__(self,index):
        return self.x[index],self.y[index]

    def __len__(self):
        return self.len

In [50]:
traindataset = my_ds(x_train,y_train)
testdataset = my_ds(x_test,y_test)

In [51]:
print(traindataset.x.size())
print(traindataset.y.size())
print(testdataset.x.size())
print(testdataset.y.size())

torch.Size([354, 13])
torch.Size([354, 1])
torch.Size([152, 13])
torch.Size([152, 1])
